
# Fine-tuning T5 Summarization

---

In [ ]:
!nvidia-smi

Tue Apr 12 20:39:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup

---

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 6.5 MB 46.5 MB/s 
     |████████████████████████████████| 895 kB 54.2 MB/s 
     |████████████████████████████████| 596 kB 74.1 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 3.3 MB/s 
     |████████████████████████████████| 212 kB 50.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.4 MB/s 
     |████████████████████████████████| 136 kB 74.1 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
     |████████████████████████████████| 144 kB 72.8 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 59.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Model and tokenizer

---

In [ ]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining, AutoModelForSeq2SeqLM, AutoModelForMaskedLM
import torch
#Llamado del modelo

#Definición de modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-arxiv")

# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-bigpatent")

# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")
# model = AutoModelForPreTraining.from_pretrained("google/bigbird-roberta-base")

# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-large")
# model = AutoModelForMaskedLM.from_pretrained("google/bigbird-roberta-large")

# Se fijan los parámetros del modelo
model.config.max_length = 500
model.config.min_length = 350
print(model.config)

# tokenización
encoder_max_length = 256 
decoder_max_length = 64

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

BigBirdPegasusConfig {
  "_name_or_path": "google/bigbird-pegasus-large-arxiv",
  "activation_dropout": 0.0,
  "activation_function": "gelu_new",
  "architectures": [
    "BigBirdPegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 2,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "length_penalty": 0.8,
  "max_length": 500,
  "max_position_embeddings": 4096,
  "min_length": 350,
  "model_type": "bigbird_pegasus",
  "num_beams": 5,
  "num_hidden_layers": 16,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "scale_embedding": t

In [ ]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='google/bigbird-pegasus-large-arxiv', vocab_size=96103, model_max_len=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})


## Data

---

### Descarga y Preparación de los Datos

### Cargado de Dataset

In [ ]:
train_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="train[:5000]")
validation_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="test[:500]")

No config specified, defaulting to: gov_report_summarization_dataset/document


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset gov_report_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f. Subsequent calls will reuse this data.


No config specified, defaulting to: gov_report_summarization_dataset/document
Reusing dataset gov_report_summarization_dataset (/root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f)


**Preprocess and tokenize**

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["report"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, 10, 10
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, 10, 10
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training

---

### Metrics

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=8,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    learning_rate=3e-04,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, #Para métricas ROUGE
    logging_dir="logs",
    logging_steps=1000,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

Wandb integration

In [ ]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="BigBird_GovReport_FT",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "GovReport",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + current_time

wandb: Currently logged in as: bigbam (use `wandb login --relogin` to force relogin)


Evaluate before fine-tuning

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 500
  Batch size = 4
Attention type 'block_sparse' is not possible if sequence_length: 10 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_gen_len': 497.95,
 'eval_loss': 9.65316390991211,
 'eval_rouge1': 1.5402,
 'eval_rouge2': 0.082,
 'eval_rougeL': 1.4954,
 'eval_rougeLsum': 1.49,
 'eval_runtime': 1795.9682,
 'eval_samples_per_second': 0.278,
 'eval_steps_per_second': 0.07}

Train the model

In [ ]:
torch.cuda.is_available()
import tensorflow as tf

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# %%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5000
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 12500


Step,Training Loss
50,9.417200
100,7.673300
150,6.957600
200,6.869000
250,6.354900
300,6.171100
350,5.669200
400,5.482000
450,5.439300
500,5.123500


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in results/checkpoint-1500/special_tokens_map.json
Saving model check

TrainOutput(global_step=12500, training_loss=2.8268662573242187, metrics={'train_runtime': 3731.7893, 'train_samples_per_second': 13.398, 'train_steps_per_second': 3.35, 'total_flos': 1410281472000000.0, 'train_loss': 2.8268662573242187, 'epoch': 10.0})

Evaluate after fine-tuning

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 500
  Batch size = 4


{'epoch': 10.0,
 'eval_gen_len': 351.0,
 'eval_loss': 5.989758014678955,
 'eval_rouge1': 5.2057,
 'eval_rouge2': 0.7879,
 'eval_rougeL': 4.8328,
 'eval_rougeLsum': 4.8711,
 'eval_runtime': 1179.394,
 'eval_samples_per_second': 0.424,
 'eval_steps_per_second': 0.106}

In [ ]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [ ]:
from transformers import BigBirdTokenizer, BigBirdModel
import torch
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["report"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = BigBirdModel.from_pretrained('google/bigbird-pegasus-large-arxiv')
test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/google/bigbird-pegasus-large-arxiv/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0c5c2a21485ba0e75fd41928cbb901586887479c8fad3f3965b9bcae7632825b.c65855e5554b00a37b55e85d3a9f9dd66ca2c3f276ee79e8daea2165fe581bbf
You are using a model of type bigbird_pegasus to instantiate a model of type big_bird. This is not supported for all configurations of models and can yield errors.
Model config BigBirdConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu_new",
  "architectures": [
    "BigBirdPegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 2,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_head

AttributeError: ignored

In [ ]:
def generate_answer(batch):
  inputs_dict = tokenizer(batch["report"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
  inputs_dict = {k: inputs_dict[k].to('CUDA') for k in inputs_dict}
  predicted_abstract_ids = model.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
  batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
  print(batch["predicted_abstract"])
  return batch

In [ ]:
dataset_small = train_data_txt
summaries_after_tuning = dataset_small.map(generate_answer)
summaries_after_tuning

  0%|          | 0/5000 [00:00<?, ?ex/s]

RuntimeError: ignored

In [ ]:
model_before_tuning = BigBirdModel.from_pretrained('google/bigbird-roberta-base').cuda()

def generate_answer(batch):
  inputs_dict = tokenizer(batch["report"], padding="max_length", max_length=255, return_tensors="pt", truncation=True)
  inputs_dict = {k: inputs_dict[k].to('cuda') for k in inputs_dict}
  predicted_abstract_ids = model_before_tuning.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
  batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
  print(batch["predicted_abstract"])
  return batch

loading configuration file https://huggingface.co/google/bigbird-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/d7643b757353be56f05bdd19496d6e3fb5bb9edfdf5f9e5eca88d6f479e32324.dc98375bb3e19a644a5cadd5c305949ec470186fcc20bd8c8b959a43dcc3ff21
Model config BigBirdConfig {
  "architectures": [
    "BigBirdForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 66,
  "tr

Downloading:   0%|          | 0.00/489M [00:00<?, ?B/s]

In [ ]:
dataset_small = train_data_txt

summaries_before_tuning = dataset_small.map(generate_answer)

In [ ]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
        ),
        headers=["Id", "Summary after"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["text"])), headers=["Id", "Document"]))